In [1]:
!python -m DMXEnttecPro.utils

COM3
  name: COM3
  description: USB Serial Port (COM3)
  hwid: USB VID:PID=0403:6001 SER=EN394342A
  vid: 1027
  pid: 24577
  serial_number: EN394342A
  location: None
  manufacturer: FTDI
  product: None
  interface: None



In [2]:
from fixtures import *
import ipywidgets as widgets
from IPython.display import display
from DMXEnttecPro import Controller

In [3]:
par_lights = Fixture (
    name="par",
    channel_count=7,
    channels=[
        Channel(1, 255, "brightness", ""),
        Channel(2, 0, "red", ""),
        Channel(3, 0, "green", ""),
        Channel(4, 0, "blue", ""),
        Channel(5, 0, "rgb", ""),
        Channel(6, 0, "strobe", ""),
        Channel(7, 0, "mode", "")
    ]
)

strobe = Fixture(
    name="strobe", 
    channel_count=6, 
    channels=[
        Channel(1, 255, "brightness", ""),
        Channel(2, 0, "strobe", "0-5 open, rest are effects, 128-250 is standard range"),
        Channel(3, 0, "red", ""),
        Channel(4, 0, "green", ""),
        Channel(5, 0, "blue", ""),
        Channel(6, 0, "auto_sound", "")
    ],
    strobe_range=(128,250)
)

fog_machine = Fixture(
    name="fog machine",
    channel_count=7,
    channels=[
        Channel(7, 0, "fog", ""),
        Channel(8, 0, "red", ""),
        Channel(9, 0, "green", ""),
        Channel(10, 0, "blue", ""),
        Channel(11, 0, "strobe", ""),
        Channel(12, 0, "stupid", "NO THIS IS A LIE, its some color changing value: 0-9 is off, can be adjusted 10-250, 250+ is music"),
        Channel(13, 0, "auto_sound", "")
    ]
)

led_bars = Fixture(
    name="led_bar",
    channel_count=432,
    channels = generate_led_bar_channels(1),
    color_max = 100
)

mirror_spotlights = Fixture(
    name="spotlights",
    channel_count=9,
    channels = [
        Channel(1, 0, "pan", ""),
        Channel(2, 0, "tilt", ""),
        Channel(3, 0, "rgb", ""),
        Channel(4, 0, "gobo", "this is a pattern, 254 is sound active slow, 255 sound active fast"),
        Channel(5, 0, "strobe", "soundactive 32-47, slow-fast is 48-239"),
        Channel(6, 0, "mirror_speed", "slow->fast"),
        Channel(7, 0, "mirror_movement", ""),
        Channel(8, 0, "laser", ""),
        Channel(9, 0, "special", "")
    ],
    strobe_range=(48,239)
)

In [4]:
colors = [
    Color("white", red=255, green=255, blue=255, spotlight_num=1),
    Color("green", red=0, green=255, blue=0, spotlight_num=10),
    Color("red", red=255, green=0, blue=0, spotlight_num=18),
    Color("blue", red=0, green=0, blue=255, spotlight_num=26),
    Color("yellow", red=255, green=255, blue=0, spotlight_num=34),
    Color("pink", red=255, green=105, blue=181, spotlight_num=42),
    Color("dark aqua", red=0, green=255, blue=255, spotlight_num=50),
    Color("light magenta", red=237, green=8, blue=237, spotlight_num=58),
    Color("sky blue", red=135, green=206, blue=250, spotlight_num=68),
    Color("magenta", red=204, green=0, blue=204, spotlight_num=74),
    Color("violet", red=237, green=130, blue=237, spotlight_num=82),
    Color("purple", red=160, green=48, blue=255, spotlight_num=90),
    Color("canary yellow", red=255, green=204, blue=17, spotlight_num=98),
    Color("lime green", red=0, green=205, blue=50, spotlight_num=106),
    Color("orange", red=255, green=165, blue=0, spotlight_num=114),
    Color("light aqua", red=0, green=219, blue=255, spotlight_num=122),
]

In [5]:
all_lights = Universe([], Controller('COM3'))
all_lights.add_fixture(par_lights)
# all_lights.add_fixture(strobe)
# all_lights.add_fixture(led_bars)
# all_lights.add_fixture(fog_machine)

In [8]:
###################BLACKOUT##################
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
title = widgets.HTML(
    value="<h1>BLACKOUT</h1>"
)

display(title)
blackout_button = widgets.Button(description="BLACKOUT")
display(blackout_button)

def on_blackout(button):
    all_lights.blackout()
    
blackout_button.on_click(on_blackout)

###########STATIC COLOR###############
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
title = widgets.HTML(
    value="<h1>SET A STATIC COLOR</h1>"
)

display(title)
color_names = []
for color in colors:
    color_names.append(color.name)

color_buttons = widgets.ToggleButtons(
    options=color_names,
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(color_buttons)

def on_click_static(button):
    all_lights.set_all_colors(next(color for color in colors if color.name == button.new))

color_buttons.observe(on_click_static, 'value')


########DRIFT##############
cancel_future__drift_calls = None
random_prior_index = 0

seconds = widgets.BoundedIntText(
    value=4,
    min=1,
    max=60,
    step=1,
    description='Seconds:'
)

seconds_pause = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=60,
    step=0.5,
    description='pause sec:'
)

display(out)
title = widgets.HTML(
    value="<h1>START CONTINuOUS COLOR DRIFT</h1>"
)

display(title)
display(seconds)
display(seconds_pause)

drift_button = widgets.Button(description="Start color drift")
drift_stop_button = widgets.Button(description="STOP")

display(drift_button)
display(drift_stop_button)
def on_drift_button_clicked(b):
    drift_button.disabled = True
    seconds.disabled = True
    drift_stop_button.disabled = False
    global cancel_future__drift_calls    
    cancel_future__drift_calls = call_repeatedly(seconds.value + seconds_pause.value, all_lights.color_fade, colors, seconds.value, True)
    
    
def on_drift_stop_button_clicked(b):
    drift_button.disabled = False
    cancel_future__drift_calls()
    drift_stop_button.disabled = True
    seconds.disabled = False

drift_button.on_click(on_drift_button_clicked)
drift_stop_button.on_click(on_drift_stop_button_clicked)

out = widgets.Output(layout={'border': '1px solid black'})
display(out)

###########COLOR CYCLE ###############
title = widgets.HTML(
    value="<h1>START CONTINuOUS COLOR CHANGE</h1>"
)

display(title)
cancel_future_calls = None
bpm = widgets.BoundedIntText(
    value=128,
    min=60,
    max=500,
    step=1,
    description='BPM:',
    disabled=False
)

display(bpm)

button = widgets.Button(description="Start color cycle")
stop_button = widgets.Button(description="STOP", disabled=True)

display(button)
display(stop_button)
def on_button_clicked(b):
    button.disabled = True
    bpm.disabled = True
    global cancel_future_calls
    cancel_future_calls = call_repeatedly(1/(bpm.value/60), all_lights.cyle_thru_colors, colors, 1)
    stop_button.disabled = False
    
def on_stop_button_clicked(b):
    button.disabled = False
    cancel_future_calls()
    stop_button.disabled = True
    bpm.disabled = False

button.on_click(on_button_clicked)
stop_button.on_click(on_stop_button_clicked)

display(out)

###############DRITFT TO COLOR###############
title = widgets.HTML(
    value="<h1>DRIFT ONCE TO COLOR</h1>"
)

display(title)
color_names = []
for color in colors:
    color_names.append(color.name)

color_buttons_fade = widgets.ToggleButtons(
    options=color_names,
    description='Fade to this color:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
duration = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description='Seconds:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(duration)
display(color_buttons_fade)

def on_click_fade(button):
    all_lights.color_fade([next(color for color in colors if color.name == button.new)], duration.value)

color_buttons_fade.observe(on_click_fade, 'value')

display(out)

##################LED BAR CHASE TO COLOR#####################################################
title = widgets.HTML(
    value="<h1>USE LED BARS TO CHASE TO COLOR</h1>"
)

display(title)
color_names = []
for color in colors:
    color_names.append(color.name)

color_buttons_led = widgets.ToggleButtons(
    options=color_names,
    description='ChaseLED to this color:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)
duration_led = widgets.IntSlider(
    value=2,
    min=1,
    max=60,
    step=1,
    description='Seconds:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(duration_led)
display(color_buttons_led)

def on_click_chase(button):
    all_lights.led_chase_to(next(color for color in colors if color.name == button.new), duration_led.value)

color_buttons_led.observe(on_click_chase, 'value')
display(out)

#####################STROBE###############################
title = widgets.HTML(
    value="<h1>SET STROBE PERCENT</h1>"
)

display(title)
color_names = []
for color in colors:
    color_names.append(color.name)

duration_strobe = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Percent:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


start_strobe = widgets.Button(description="Start strobe")
stop_strobe = widgets.Button(description="Stop strobe")

display(start_strobe)
display(stop_strobe)
display(duration_strobe)

def on_click_strobe(button):
    all_lights.set_strobe_percent(duration_strobe.value)
    
def on_click_stop_strobe(button):
    all_lights.set_strobe_percent(0)
    
stop_strobe.on_click(on_click_stop_strobe)
    
start_strobe.on_click(on_click_strobe)

display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

HTML(value='<h1>BLACKOUT</h1>')

Button(description='BLACKOUT', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

HTML(value='<h1>SET A STATIC COLOR</h1>')

...

HTML(value='<h1>SET STROBE PERCENT</h1>')

Button(description='Start strobe', style=ButtonStyle())

Button(description='Stop strobe', style=ButtonStyle())

IntSlider(value=50, continuous_update=False, description='Percent:')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
for fixture in all_lights.fixtures:
    if fixture.name != "led_bars":
        print(f'\n{fixture.name}:\n')
        for channel in fixture.channels:
            print(channel)


par:

Channel(index=1, value=255, type='brightness', description='')
Channel(index=2, value=255, type='red', description='')
Channel(index=3, value=165, type='green', description='')
Channel(index=4, value=0, type='blue', description='')
Channel(index=5, value=0, type='rgb', description='')
Channel(index=6, value=0, type='strobe', description='')
Channel(index=7, value=0, type='mode', description='')


In [16]:
all_lights.set_channel_value(channel_num=7, value=0)

In [9]:
from DMXEnttecPro import Controller
dmx = Controller('COM6')  # Typical of Windows
# dmx = Controller('/dev/ttyUSB0')  # Typical of Linux

SerialException: could not open port 'COM6': PermissionError(13, 'Access is denied.', None, 5)

In [ ]:
# dmx.set_channel(1, 255)  # 
# dmx.set_channel(2, 255)  #
# dmx.set_channel(3, 63)  #
# dmx.set_channel(4, 0)  #


dmx.set_channel(1, 0)  # fog YES
dmx.set_channel(2, 255)  #RED
dmx.set_channel(3, 255)  #GREEN
dmx.set_channel(4, 255) #BLUE
dmx.set_channel(5, 0)  # Strobe
dmx.set_channel(6, 0) # COLOR CHANGE?
dmx.set_channel(7, 200)  # sound control YES
dmx.submit()  # Sends the update to the controller

In [ ]:
dmx.close()